# DENUE Exploratory Data Analysis notebook

[DENUE](http://www.beta.inegi.org.mx/app/mapa/denue/) means Directorio Estadístico Nacional de Unidades Economicas. It is maintained by [INEGI](http://www.inegi.org.mx/), the Mexican Institute of Statistics and Geography.

In this notebook we start exploring the data as part of our project to map the creative industries in Mexico

## About the data

This [methodological note](http://www.beta.inegi.org.mx/app/biblioteca/ficha.html?upc=702825097240) describes the data (in Spanish).

DENUE contains information about ~5 million active businesses in Mexico. This includes:

* Identification
* Location
* Sector
* Size

DENUE’s first edition was published in 2010, with data collected in 2009. Individual entrepreneurs and organisations are legally required to register in DENUE and update their information. Other parts of the public sector are also required to provide INEGI with data to maintain DENUE. 

**Data updates**

* Information about larger businesses (>20 employees / $20m in turnover or activities in more than one State) as well as those in some sectors are updated yearly with administrative data and business surveys.

* Information about micro/small/medium businesses is partially updated through administrative registers.

* Users update their information online continuously, and this information is quality assured by INEGI.

* All data in DENUE are updated every 5 years through information produced by the economic census. The latest complete update of DENUE was in 2016, using data from the 2014 economic census.

### Conceptual framework and coverage

**Unit of analysis**

DENUE contains information about establishments (local units) and enterprises. 

**Sector**

DENUE contains information about all industries with the exception of agriculture, farming and forestry, transport, political organisations, domestic workers and extraterritorial units.

Industrial activities are classified according to 2013 NAICS. The first edition of DENUE using 2013 NAICs was in 2015 (*this means we can probably use definitions of the creative industries implemented by Nesta in its analysis of the creative industries in North America*)

**Period**

The period covered by business data depends on the year of incorporation.
  * For businesses incorporated before 2014, DENUE data refers to 2014
  * For businesses incorporated Jan 2015- Oct 2016 - DENUE data refers to 2015
  * For business incorporated Mar 2017-Nov 2017 - Data refers to 2016

**Geography**

DENUE includes economic units in locations with 2500+ inhabitants as well as ‘economic locations’ such as industrial parks. It doesn’t contain economic units in rural areas./

Data are available at the following levels of resolution:
* AGEE (States) - 2 digit based on State name. Administrative boundary
* AGEM (towns / cities) - 3 digits.
* AGEB (micro-geography):
  * Urban: 1-50 squares
  * Rural: ~11m Ha.

Longitude and Latitude data are also available. 

### Variables

* Nombre: Name. Some organisations have generic names.
* Denominación o razón social y tipo de sociedad: Type of organisation
* Código: industrial code based on NAICs (organisations are allocated to the sector which generates most turnover for the company)
* Personal: 8 sizebands (0-251+)
* Tipo de Unidad económica: 
  * Whether the organisation is a subsidiary, and the type of establishment.
* Datos de ubicación: Location (address)
* Area geográfica: (see above)
* Contact details:
  * Telephone
  * Web address
  * Email



## Some processing activities

* Load data
* Merge with creative NAICs codes
* Produce some descriptive analysis: creative sectors by location (totals, specialisation)
* Analyse access to contact details




## Preamble

This includes:

* Package imports
* Paths
* Functions and classes used below



In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import datetime
import pandas as pd
import geopandas as gp
import numpy as np
import zipfile
import os
import io
import requests
import re

today = datetime.datetime.today()
today_str = "_".join([str(x) for x in [today.day,today.month,today.year]])

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
denue_dir = os.path.join(parent_dir, 'data', 'denue')
nesta_dir = os.path.join(parent_dir, 'data', 'nesta') 
processed_dir = os.path.join(parent_dir, 'data', 'processed')
shapefile_dir = os.path.join(parent_dir, 'data', 'shapefiles') 
reports_dir = os.path.join(parent_dir, 'reports')
fig_dir = os.path.join(parent_dir, 'reports', 'figures')

## Download data

In [2]:
denue_listado = open(os.path.join(os.getcwd(), 'DescargaMasivaOD.xml'), 'r')
parsed = BeautifulSoup(denue_listado, "xml")
urls = [x.text for x in parsed.find_all('Archivo')]
files_dl = [x for x in urls if re.search(r'denue_00_[0-9\-]+_csv', x)]
for f in files_dl:
    print(f)

https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_11_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_21_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_22_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_23_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_31-33_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_43_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46111_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46112-46311_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46321-46531_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_46591-46911_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_48-49_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_51_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_52_csv.zip
https://www.inegi.org.mx/contenidos/masiva/denue/denue_00_53_csv.zi

In [3]:
for url in files_dl:
    dl = requests.get(url)
    zf = zipfile.ZipFile(io.BytesIO(dl.content))
    with open(os.path.join(denue_dir, url.split('/')[6]), 'wb') as outfile:
        outfile.write(dl.content)
    print(f'{url} descargado en {denue_dir}')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/enrique/Documents/mexico_ci_mapping/data/denue/denue_00_11_csv.zip'

## 1. Load data

Here we will process all relevant DENUE files. There are quite a few of them and they are big. To deal with this I will write a class that loads them one at a time, unzips them, extracts companies in creative sectors (based on a lookup table based on NAICS codes) and generates some sector / area reports to normalise findings.


### a. Load metadata

Lookup between NAICs and creative industries codes based on Nesta 2016 (see Table A10 in /references folder)

Some of the codes in that table are too coarse (ie advertising also includes management consultancy). We have created a more refined classification that removes some of the non-creative subcodes, but we will generally work with Nesta's classification for consistency. It would be easy to replicate the analysis focusing on the refined categories


In [ ]:
#Load a file with NAICs codes
naics_creative_lookup = pd.read_csv(os.path.join(nesta_dir, "naics_creative_short.csv"))
naics_creative_lookup.head()

Load shapefiles for mapping. We downloaded them from [here](http://www.inegi.org.mx/geo/contenidos/geoestadistica/m_geoestadistico.aspx).

In [ ]:
estados = gp.read_file(os.path.join(shapefile_dir, "mexico"))

#They were easy to load. We can use them below
estados.head()

In [ ]:
estados.plot()

### b. Load one file for exploration

In [ ]:
#This creates the file. We will load one that has creative sectors in it
zf = zipfile.ZipFile(os.path.join(denue_dir, 'denue_00_51_csv.zip'))

#Extract the names
zf_names = zf.infolist()

for f in zf_names:
    print(f.filename)

#### Load the data dictionary

In [ ]:
#Denue data dictionary

#This gets the filename for the data dictionary
data_dict_filename = zf_names[0].filename

#And we read it
denue_dictionary = pd.read_csv(zf.open(data_dict_filename), encoding='latin-1')

#We save it so we can have a look at it, and maybe create cleaner names and variables
denue_dictionary.to_csv(os.path.join(processed_dir, '{date}_denue_data_dictionary.csv'.format(date=today_str)))

#### Load the data

In [ ]:
#We are interested in the 'denue_inegi_51_.csv' file, index=1
denue_data = zf_names[1]

#We get its name (which we will use to open the data)
denue_file_name = denue_data.filename

In [ ]:
#Pandas can read this easily
denue_51 = pd.read_csv(zf.open(denue_file_name), encoding='latin-1')

In [ ]:
#Here it is
denue_51.head()

In [ ]:
#It contains 19K observations and 41 variables
denue_51.shape

In [ ]:
#These are the top 10 sectors
denue_51.nombre_act.value_counts()[:10]

In [ ]:
#What do the sector codes look like? Can we find them in the NAICs lookups

list(set(denue_51.codigo_act))[:5]

They have 7 digits

In [ ]:
#Check lookup. We do this by dropping duplicate codes in the data
code_name_lu = denue_51[['codigo_act','nombre_act']].drop_duplicates('codigo_act').reset_index(drop=True)

#We have compared the codes in the data with the codes in the NAICs file and they match.
#We will work with the NAICs file, enriched with the codes from Nesta (2016) report

In [ ]:
denue_51.telefono

### Merge DENUE and lookup

In [ ]:
#Let's match this with the metadata to see what we get
denue_51_creative = pd.merge(denue_51,naics_creative_lookup,left_on='codigo_act',
                            right_on='code_int',how='left')

#Look at software companies
denue_51_creative.loc[denue_51_creative.creative_sector=='software',:].head()

In [ ]:
#Creative sectors in the data
denue_51_creative.creative_sector.value_counts()

In [ ]:
#Sizes
pd.crosstab(denue_51_creative.per_ocu,denue_51_creative.creative_sector)

In [ ]:
#States
pd.crosstab(denue_51_creative.entidad,denue_51_creative.creative_sector)

In [ ]:
#Municipios
pd.crosstab(denue_51_creative.municipio,denue_51_creative.creative_sector)[:20]

## c. Load all data

**Plan**

I will write a class that takes a file name, reads it, extracts all creative businesses and also the distribution of companies in all other areas, as well as their size distribution (which we can use to normalise the data)

This involves:

1. Creating a list of all filenames I want to load
2. Writing a class with: 
 * A read method to read the CSV and rename the variables to something tidier
 * A get_creative method that merges the file with naics_creative lookup and returns all matches
 * An area_stats method that generates company counts by state and municipio
3. Implementing the class while I go for a run :-)


In [ ]:
class ReadProcessDenue():
    
    '''
    DenueProcess is a class that takes a filename for a zip file and:
   
    -Extracts the csv content (read method)
    -Cleans variable names (process method)
    -Merges with the NAICS Lookup (get_creative method)
    -Generates area counts for all sectors (area_counts method).
    
    It stores the outputs in a creative_businesses attribute, a state_counts attribute and a local_counts 
    attribute. 

    '''
    
    def __init__(self,filepath):
        '''
        Initialise the class with a filepath
        
        '''
        
        self.filepath = filepath
        
    def read_file(self):
        '''
        Read the csv
        
        '''
        #Get the filepath
        filepath = self.filepath
        
        #Read the zipfile
        zf = zipfile.ZipFile(filepath)
        
        #Get the name of the datafile
        data_file_name = zf.infolist()[1].filename
        
        #Read the file
        denue_data = pd.read_csv(zf.open(data_file_name), encoding='latin-1')
        
        #Assign the denue data to the denue_data attribute
        self.denue_data = denue_data

    
    def get_creative(self):
        '''
        Extracts creative companies
        
        '''
        #Read all denue data
        all_denue = self.denue_data
        
        #Merge with the creative lookup
        creative_denue = pd.merge(all_denue,naics_creative_lookup,
                                  left_on='codigo_act',right_on='code_int',how='inner').reset_index(drop=True)
        
        #Convert the telephone number to a string
        #NB the if else is to deal with missing values in the telefone field
        #creative_denue['telefono'] = [str(int(x)) if type(x)!=float else np.nan for x in creative_denue['telefono']]
        
        
        #We will store two versions: a complete file and a short file with less variables of interest
        
        #Complete version with somewhat messy variable names
        self.denue_creative_long = creative_denue
        
        #Also store the tidy version
        #NB we need to re-concatenate it with the sector info we merged above
        self.denue_creative_short = pd.concat([tidy_denue_data(creative_denue),
                                               creative_denue[['creative_nesta_2016','creative_refined',
                                                               'creative_sector','has_creative']]],axis=1)
                                               
        
        
                                               
    def get_context_stats(self):
        '''
        This extracts contextual information that we will use to compare and normalise the creative data.
        
        NB this is only preliminary. I assume we will want to do more finely grained comparisons. We
        can modify this method to extract what we need to do that.
        
        For now we will get:
        -State counts
        -Municipio counts
        -Sizeband counts
        -Sizeband by state and municipality counts
        -Incorporation date counts
        
        '''
        
        #Load all denue data
        all_denue = self.denue_data
        
        #Extract contextual reports with a function
        self.context_stats = get_context(all_denue)
        
        
        
        

In [ ]:
def tidy_denue_data(denue_data):
    '''
    This function returns a clean denue file with a few relevant variables
    We will use this for economic analysis
    NB the variables we are removing will nevertheless be important for marketing
    
    '''
    
    interesting_vars = ['id','nom_estab','raz_social','codigo_act','nombre_act','per_ocu',
                       'cod_postal','entidad','municipio','localidad','telefono','correoelec','www',
                       'tipoUniEco','latitud','longitud','fecha_alta']
    
    tidy_name_vars = ['id','name','legal_name','sector','sector_description','employee_sizeband',
                      'postcode','state','municipality','location','telephone','email','website','type_organisation',
                      'lat','lon','incorp_date']
    
    denue_selected = denue_data[interesting_vars]
    
    denue_selected.columns = tidy_name_vars
    
    return(denue_selected)

def get_context(denue_data):
    '''
    This function extracts some contextual data as part of the get_context_stats method in the ReadProcessDenue
    class.
    
    '''
    
    #Tidy data (having standard names will help processing later)
    tidy_denue = tidy_denue_data(denue_data)
    
    #Generate a bunch of counts
    state_counts, municipality_counts,sizeband_counts,incorp_date_counts = [tidy_denue[x].value_counts() for x in 
                                                         ['state','municipality',
                                                          'employee_sizeband','incorp_date']]
    
    #And a couple of crosstabs
    state_sizeb_crosstabs,municipality_sizeb_crosstabs = [pd.crosstab(tidy_denue[x],
                                                                      tidy_denue['employee_sizeband']) for x in
                                                         ['state','municipality']]
    
    #Put everything in a list
    out_list = [state_counts,municipality_counts,sizeband_counts,incorp_date_counts,
               state_sizeb_crosstabs,municipality_sizeb_crosstabs]
    
    return(out_list)
    
    
    

In [ ]:
#Let's run this

#Extract a list of files in the 'sector data' directory
all_sector_files = os.listdir(denue_dir)
sector_files_csv = [x for x in all_sector_files if 'csv' in x]


#This is a container list where we will put the info we are interested in for now (the tidy denue data and the 
# contextual data

denue_eda_outputs = []


#We'll run it as an old school loop
for dataset in sector_files_csv:
    
    #For each dataset...
    #We print it to keep track of things
    
    print(dataset)
    
    #Create the path
    input_file_path = os.path.join(denue_dir, dataset)
    
    #Initialise the class
    denue_object = ReadProcessDenue(input_file_path)
    
    #Read the data
    denue_object.read_file()
    
    #Get the creative data
    denue_object.get_creative()
    
    #Get the contextual data
    denue_object.get_context_stats()
    
    #Put everything in a list
    outputs_of_interest = [denue_object.denue_creative_short,denue_object.denue_creative_long,
                          denue_object.context_stats]
    
    #Append to DENUE
    denue_eda_outputs.append(outputs_of_interest) 
    
    

#I'm ignoring the warnings although eventually I will fix the code to remove them

In [ ]:
#This concatenates all the creative companies into a single dataframe
creative_tidy = pd.concat([x[0] for x in denue_eda_outputs]).reset_index(drop=True)
creative_tidy.to_csv(os.path.join(processed_dir, '{today}_denue_creative.csv'.format(today=today_str)))

## 2. Explore data

Where we explore the DENUE data we have downloaded. 

The exploration has 2 angles:

* Descriptives
  * Sectoral distribution
  * Geography of the sector (including maps and specialisation)
  * Incorporation dates ('age'?)
* Comparative analysis
  * Compare CI sizebands with rest of the economy or other sectors
  * Compare CI incorporation dates with rest of the economy or other sectors
* Availability of contact details
  * This will inform the survey design and implementation 


### Sectoral distribution

#### How many creative businesses are there in Mexico, and in what sectors?

In [ ]:
#According to the Nesta NAICS definition
nesta_2016_def = creative_tidy.loc[creative_tidy.creative_nesta_2016==1,:]

nesta_ref = creative_tidy.loc[creative_tidy.creative_refined==1,:]

print("According to the Nesta 2016 NAICs definition there are {num} creative business in Mexico".
      format(num=len(nesta_2016_def)))

print('\n')

print("According to the refined definition there are {num} creative business in Mexico".
      format(num=len(nesta_ref)))

In [ ]:
#Do they have unique ids? 

#Yes. We should in any case check their legal status
len(set(nesta_2016_def.id))

In [ ]:
#What percentage of the total does this represent?

In [ ]:
nesta_2016_def.loc[nesta_2016_def.website.isna()==False,:][
    ['id','name','creative_sector','employee_sizeband','website']].to_csv(
    os.path.join(processed_dir,'denue_website_data.csv'))

In [ ]:
total_businesses_denue = np.sum([x[2][0].sum() for x in denue_eda_outputs])

#Percentage of creative businesses in DENUE
100*90004/total_businesses_denue

In [ ]:
#Compare both:

#We create a table that concatenates both sectors
creative_comp_table = pd.concat([nesta_2016_def['creative_sector'].value_counts(),
                                nesta_ref['creative_sector'].value_counts()],axis=1,sort=True)

#Rename columns
creative_comp_table.columns=['creative_nesta_2016','creative_refined_def']

#Sort values
creative_comp_table.sort_values('creative_nesta_2016',ascending=False,inplace=True)

#We extract the index of the table to sort charts later
sectors_sorted = creative_comp_table.index


fig,ax = plt.subplots(figsize=(10,8))

creative_comp_table.plot.bar(ax=ax)


ax.set_yticklabels(ax.get_yticks(),size=14)
ax.set_xticklabels(labels=creative_comp_table.index,size=16,rotation=45,ha='right')
ax.legend(fontsize=14)
ax.set_title('Number of creative businesses in DENUE according to different definitions',fontsize=16)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_sector_bar_chart.pdf'.format(date=today_str)))

If we look at the `naics_creative_lookup_short` file we can see what explains the differences:

* Music and performing arts in Nesta 2016 includes many generic and sports related educational activities
* Advertising and marketing in Nesta 2016 includes management consulting activities
* Architecture in Nesta 2016 includes engineering and drafting services, building inspection and other things unrelated to architectural design
* Crafts in Nesta 2016 doesn't include several activities related to jewelry and metalwork

As mentioned before, we will focus on the Nesta definition for now. We can change the definition later.

#### How many people work in the sector?

In [ ]:
#We call them cis for creative industries
cis = nesta_2016_def.copy()

#Make estado lowercase to match with the map data later
cis['state'] = [x.lower().strip() for x in cis['state']]


In [ ]:
#This is the size distribution

size_distribution = pd.DataFrame(cis['employee_sizeband'].value_counts())
size_distribution['share_pc'] = size_distribution/size_distribution.sum()

sizeb_sorted = size_distribution.index

size_distribution

In [ ]:
#Sizeband by creative subsector compared with the creative industries overall and other sectors

sector_size = pd.crosstab(cis['creative_sector'],cis['employee_sizeband'],normalize=0)


#Now we combine with the size distribution chart above and with a size distribution table *for all companies*

#This creates a sizeband distribution for all businesses in DENUE
sizeband_distr_all= pd.concat([x[2][2] for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1)

#We calculate shares
sizeband_distr_shares = sizeband_distr_all/sizeband_distr_all.sum()

#Concatenate everything
sector_size_all = pd.concat([sector_size.T,size_distribution['share_pc'],sizeband_distr_shares],axis=1,sort=True)

#We rename the columns for all creative and other sectors
sector_size_all.rename(columns={'share_pc':'creative_industries',0:'all_sectors'},inplace=True)

#We want to show all creative subsectors first, sorted by their size. 


sectors_sorted = sector_size_all.T.sort_values(['0 a 5 personas'],ascending=True).index

#We put creative industries and all sectors at the end. This is what this list comprehension is for
sectors_sorted = [x for x in sectors_sorted if x not in ['all_sectors','creative_industries']]+[
    'creative_industries','all_sectors']

In [ ]:
fig,ax = plt.subplots(figsize=(12.5,8))

sector_size_all.T.loc[sectors_sorted,sizeb_sorted].plot.bar(stacked=True,ax=ax,width=0.75,
                                                      title='Size distribution by sector')

#We adjust limits to fit the legend on the right
ax.set_xlim((-0.5,14))

ax.set_yticklabels(ax.get_yticks(),size=14)
ax.set_xticklabels(labels=sectors_sorted,size=14,rotation=45,ha='right')
ax.legend(fontsize=14)
ax.set_title('Sector sizeband distribution',fontsize=16)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_size_distribution.pdf'.format(date=today_str)))



In [ ]:
#Create a midpoint estimate

In [ ]:
def get_midpoint(sizeband_value):
    '''
    This function extracts a midpoint estimate from the sizeand variable in the DENUE data
    
    '''
    
    #With the exception of '251 y mas...' the categories say 'x to y' people. We will split on whitespace
    #and extract the values,turn them into integers and average them.
    
    #We assume that the top value is 251. Maybe we could survey these companies to get their employment. Who are they?
    
    if '251' in sizeband_value:
        estimate = 251
        
    else:
        #Split
        split = sizeband_value.split(" ")

        #Get mean
        estimate = np.mean([int(split[0]),int(split[2])])
        
    return(estimate)
        
    

In [ ]:
#Here it is
cis['employment_estimate'] = [get_midpoint(x) for x in cis.employee_sizeband]

In [ ]:
cis.employment_estimate.sum()

This number is significantly lower than what other people say, eg '2m jobs including 'direct and indirect' jobs' according to [this blog](https://www.forbes.com.mx/la-cultura-riqueza-mal-vista/) (Note that it includes no definition). Note that the right-censoring of the data means that we are probably underestimating the size of the companies in DENUE (for example, according to [Wikipedia](https://es.wikipedia.org/wiki/Televisa), Televisa employs 15,000 people but these data would estimate its employmnent at 11.

Maybe we could run the bigger companies vs Wikipedia to extract their employment? What are they?

In [ ]:
large_company_list = list(cis.loc[cis.employment_estimate>250,'name'])

In [ ]:
large_company_list[:10]

#There is some repetition in the names. Are these different organisations or duplicates?

In [ ]:
#Create sector size estimates
sector_size_estimates = cis.groupby('creative_sector')['employment_estimate'].sum().sort_values(ascending=False)

fig,ax = plt.subplots(figsize=(10,5))

sector_size_estimates.plot.bar(ax=ax,color='orange',title='Sector size estimates')


ax.set_yticklabels(ax.get_yticks(),size=14)
ax.set_xticklabels(labels=creative_comp_table.index,size=14,rotation=45,ha='right')
ax.legend(fontsize=14)
ax.set_title('Sector size estimates',fontsize=16)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_size_estimates.pdf'.format(date=today_str)))

#### Comparing employment in CIs with other industries

In [ ]:
#This will concatenate business counts in different sizebands for each sector outside of the CIs

all_sizebands = pd.concat([x[2][2] for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1).reset_index(drop=False)

#We produce the 'midpoint estimate' of the data 
all_sizebands['midpoint'] = [get_midpoint(x) for x in all_sizebands['index']]

#All employment multiplies the midpoint estimate by the number of companies in each sizeband
all_employment = np.sum(all_sizebands['midpoint']*all_sizebands[0])

#We divide the creative employment by all employment to get an estimate

np.round(100*699838.0/all_employment,3)

2.5% of all estimated employment is in the creative industries

### Other inter-sectoral comparisons







#### Incorporation dates

Date when different organisations joined the register. This isn't the same as company age.

We will look at this by subsector and compared to the creative industries.



In [ ]:
#How many incorporation values are there?

cis_join_date = cis.incorp_date.value_counts()

cis_join_date.name='creative_industries'

cis_subsector_join_date = pd.crosstab(cis.incorp_date,cis.creative_sector)

#How do they compare with the data for all other sectors?
#As previously, we extract the information from the denue outputs container and sum to
#get total number of companies by date of incorporation into the register 

all_join_date = pd.concat([x[2][3] for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1)
all_join_date.name='all_sectors'


#Let's put everything together

#We want to sort the dates from the oldest to the newest
merged_join_date =pd.concat([cis_subsector_join_date,cis_join_date,all_join_date],axis=1,sort=True).reset_index(drop=False)

#Combine inclusion dates by year
merged_join_date['year'] = [int(x.split("-")[1]) for x in merged_join_date['index']]

#Sum by year and reindex
merged_join_date = merged_join_date.drop('index',axis=1).groupby('year').sum().apply(lambda x: x/x.sum(),axis=0)

#Sort sectors by 'novelty' (Importance of incorporations in 2014 - 2016) with all_sectors and creative_industries
#at the end
sectors_sorted_date = merged_join_date.T.iloc[:,-2:].sum(axis=1).sort_values(ascending=False).index

sectors_sorted_date = [x for x in sectors_sorted_date if x not in [
    'creative_industries','all_sectors']] + ['creative_industries','all_sectors']

In [ ]:
fig,ax = plt.subplots(figsize=(12.5,8))

merged_join_date.T.loc[sectors_sorted_date,:].plot.bar(stacked=True,ax=ax,width=0.75,
                                                      title='DENUE registering date by sector')

#We adjust limits to fit the legend on the right
ax.set_xlim((-0.5,12.2))


ax.set_yticklabels(ax.get_yticks(),size=14)
ax.set_xticklabels(labels=sectors_sorted_date,size=14,rotation=45,ha='right')
ax.legend(fontsize=14)
ax.set_title('Sector DENUE incorporation distribution',fontsize=16)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_incorp_distribution.pdf'.format(date=today_str)))

plt.tight_layout()

In [ ]:
#Finally, check incorporation dates by size

#Crosstab
inc_dates_size = pd.crosstab(cis.incorp_date,cis.employee_sizeband,normalize=1).loc[:,sizeb_sorted]

#As before, we extract the year from the incorporation variable so we can aggregate and sort more easily
inc_dates_size['year'] = [int(x.split("-")[1]) for x in inc_dates_size.index]


fig,ax = plt.subplots(figsize=(9,5))

inc_dates_size.groupby('year').sum().T.plot.bar(stacked=True,ax=ax,width=0.75,
                                                      title='DENUE registering date by sizeband (all creative)')

#We adjust limits to fit the legend on the right
ax.set_xlim((-0.5,7.5))

This is kind of surprising - many larger companies have been added to the data in recent years. Let's check some of them

In [ ]:
cis.loc[(['2016' in x for x in cis['incorp_date']]) & (cis.employment_estimate>250),:].head()

Overall the numbers are small (36)

#### Where is the sector?

We will examine its geography in terms of:
* Total levels of activity by state and municipality, and totals by subsector
* Values normalised by activity in other industries

In [ ]:
cis.groupby('state')['employment_estimate'].sum()

In [ ]:
#Let's end the day with a map

#Distribution of businesses by state
state_creative_counts = cis['state'].value_counts()

#Employment estimates
state_employment_estimates = cis.groupby('state')['employment_estimate'].sum()

#Combine business and employment estimates
state_all_creative = pd.concat([state_creative_counts,state_employment_estimates],axis=1,sort=True)

#Remember we loaded the estado shapefile before. We make the estate name lowercases to merge on them
estados['ESTADO'] = [x.lower().strip() for x in estados['ESTADO']]

#They also use different names for Mexico City (DENUE calls it distrito federal and 
#the shapefiles call it Ciudad de Mexico. 
#We fix hackily

# Shapefile state names need to coincide with denues names.
estados['ESTADO'] = ['michoacán de ocampo' if x =='michoacán' else x for x in estados['ESTADO']]
estados['ESTADO'] = ['veracruz de ignacio de la llave' if x =='veracruz' else x for x in estados['ESTADO']]
estados['ESTADO'] = ['ciudad de méxico' if x =='distrito federal' else x for x in estados['ESTADO']]
estados['ESTADO'] = ['coahuila de zaragoza' if x =='coahuila' else x for x in estados['ESTADO']]

#Merge them
state_creative_polys = pd.merge(estados,state_all_creative.reset_index(drop=False),left_on='ESTADO',
                               right_on='index')

state_all_creative
estados

In [ ]:
fig,ax = plt.subplots(figsize=(13,7))

#Plot business counts
state_creative_polys.plot('state',ax=ax,cmap='viridis',legend=True)

ax.set_title('Number of creative businesses by state',size=16)

#Set axis off
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{today}_total_business_map.png'.format(today=today_str)))


In [ ]:
fig,ax = plt.subplots(figsize=(13,7))

#Plot business counts
state_creative_polys.plot('employment_estimate',ax=ax,cmap='viridis',legend=True)

ax.set_title('Creative employment level by state',size=16)

#Set axis off
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)


plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{today}_employment_estimate_map.png'.format(today=today_str)))

In [ ]:
#What share do the top 5 locations represent for creative business counts and employment
print(state_creative_counts.sort_values(ascending=False)[:5]/state_creative_counts.sum())
print(state_employment_estimates.sort_values(ascending=False)[:5]/state_employment_estimates.sum())

#### Create a chart that measures the levels of concentration in different creative sectors

In [ ]:
#We could probably put this function in the previous class to keep things tidy
def get_state_employment_estimates(df):
    '''
    This function extracts, for each state-size distribution table,
    an estimate of employment based on company sizebands
    '''
    
    #We melt the dataframe to simplify processing
    #We need to reset the index to use it as the id var when melting
    df_melted = pd.melt(df.reset_index(drop=False),id_vars='state')
    
    df_melted['midpoint'] = [get_midpoint(x) for x in df_melted['employee_sizeband']]
    
    #Multiply the midpoint by number of businesses
    df_melted['employment_estimate'] = df_melted['midpoint']*df_melted['value']
    
    #Regroup
    df_aggregate = df_melted.groupby('state')['employment_estimate'].sum()
    
    return(df_aggregate)


def lorenz_plot(shares_df,name_for_title,ax=ax):
    '''
    Function that takes a dataframe with shares of activity by observation and returns 
    a figure plotting them buy ranking.
    
    '''
    #Loops over every column, sorts it and plots it. We play with colours to highlight creative
    #and all_sectors
    
    for x in shares_df.columns:
        ax.plot(np.array(shares_df.loc[:,x].sort_values(ascending=False)),
                c='red' if x=='all_creative' else 'black' if x=='all_sectors' else sector_map_lookup[x],
                linewidth=4 if x=='all_creative' else 4 if x=='all_sectors' else 3,
           #alpha=0.9 if x not in ['all_sectors','all_creative'] else 1
               )

    ax.legend(labels=shares_df.columns,fontsize=14)
    ax.set_title("{name}:State share of total by state ranking".format(name=name_for_title),size=16)

    ax.set_yticklabels([100*x for x in ax.get_yticks()],size=12)
    ax.set_xticks(np.arange(len(shares_df)))
    ax.set_xlabel('State position',fontsize=12)  
    ax.set_ylabel('% of all {name} activity'.format(name=name_for_title),fontsize=14)  
    
    return(ax)
    
    

In [ ]:
#Are the creative industries more concentrated than other sectors? We look at this based on 


# Get sectoral shares, creative shares and total shares
creative_subsector_shares = pd.crosstab(cis.state,cis.creative_sector, normalize=1)
creative_industries_shares = cis.state.value_counts(normalize=True)
creative_industries_shares.name = 'all_creative'

#All sectors
all_sectors_state_counts = pd.concat([x[2][0] for x in denue_eda_outputs],sort=True).reset_index(
    drop=False).groupby('index')['state'].sum()
all_sectors_state_shares = all_sectors_state_counts/all_sectors_state_counts.sum()
all_sectors_state_shares.index = [x.lower().strip() for x in all_sectors_state_shares.index]
all_sectors_state_shares.name = 'all_sectors'

#All state shares
all_state_shares = pd.concat([creative_subsector_shares,creative_industries_shares,
                              all_sectors_state_shares],axis=1,sort=True)

#Same thing for employment:
#Creative subsector
#Create total employment levels, pivot and normalise
creative_subsector_empl_shares = pd.pivot_table(
    cis.groupby(['state','creative_sector'])['employment_estimate'].sum().reset_index(drop=False),
    index='state',columns='creative_sector',values='employment_estimate').apply(lambda x: x/x.sum(),axis=0)

#All creative industries
all_cis_state_employment = cis.groupby(['state'])['employment_estimate'].sum()
all_cis_state_empl_shares = all_cis_state_employment/all_cis_state_employment.sum()

all_cis_state_empl_shares.name = 'all_creative'

#All economy
all_sectors_employment_state = pd.concat(
    [get_state_employment_estimates(x[2][4]) for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1)

all_sectors_employment_state.index = [x.lower().strip() for x in all_sectors_employment_state.index]

#Create shares
all_sectors_empl_state_share = all_sectors_employment_state/all_sectors_employment_state.sum()
all_sectors_empl_state_share.name = 'all_sectors'

all_state_empl_shares = pd.concat([creative_subsector_empl_shares,all_cis_state_empl_shares,
                                  all_sectors_empl_state_share],axis=1)

In [ ]:
#Finally: plot

#Plot
sector_colors = plt.cm.get_cmap('tab20_r').colors[:len(all_state_shares.columns[:-1])]

sector_map_lookup = {x:y for x,y in zip(all_state_shares.columns,sector_colors)}


fig,ax = plt.subplots(figsize=(17,9),ncols=2,sharey=True)

lorenz_plot(all_state_shares,ax=ax[0],name_for_title='Number businesses')
lorenz_plot(all_state_empl_shares,ax=ax[1],name_for_title='Employment level')

plt.tight_layout()

fig.suptitle('Concentration of creative businesses / employment by state',y=1.02,size=18)

plt.savefig(os.path.join(fig_dir,'{date}_lorenz_states.pdf'.format(date=today_str)),bbox_inches='tight')


In [ ]:
#Plot
sector_colors = plt.cm.get_cmap('tab20_r').colors[:len(all_state_shares.columns[:-1])]

sector_map_lookup = {x:y for x,y in zip(all_state_shares.columns,sector_colors)}

fig,ax = plt.subplots(figsize=(8,6))

for x in all_state_shares.columns:
    ax.plot(np.array(all_state_shares.loc[:,x].sort_values(ascending=False)),
           c=sector_map_lookup[x] if x!='all_sectors' else 'black',linewidth=3,
           alpha=0.9 if x not in ['all_sectors','all_creative'] else 1)

ax.legend(labels=all_state_shares.columns)
ax.set_title("State share of total by state ranking",size=14)

ax.set_yticklabels([100*x for x in ax.get_yticks()],size=12)
ax.set_xticks(np.arange(len(all_state_shares)))
ax.set_xlabel('State position',size=12)  
ax.set_ylabel('% of all activity',size=12)  

plt.tight_layout()

### Specialisation

In [ ]:
# Let's calculate measures of specialisation

#This function calculates LQs

#Functions
def create_lq_df(df,year=None):
    '''
    Takes a df with cells = activity in col in row and returns a df with cells = lq
    
    '''
    
    area_activity = df.sum(axis=0)
    area_shares = area_activity/area_activity.sum()
    
    lqs = df.apply(lambda x: (x/x.sum())/area_shares, axis=1)
    
    if year!=None:
        lqs['period'] = year
    
    return(lqs)

In [ ]:
def process_counts_to_get_lqs(subsector_df,creative_df,all_sectors_df):
    '''
    This function takes a subsector df, creative df and all sectors df
    and operates on them to return a creative LQ df. 
    
    '''
    
    creative_all_state = pd.concat([creative_df,all_sectors_df],axis=1)
    creative_all_state.columns=['creative_industries','all_industries']

    creative_all_state['not_creative'] =creative_all_state['all_industries'] - creative_all_state['creative_industries']
    
    
    #Now we have everything we need to calculate the LQ
    #We use the function we defined above. We combine creative data with 
    #not creative to normalise over the total in the locality.
    #NB we drop not creative at the end because we don't plan to report it.

    creative_subsector_state_lq,creative_industries_state_lq = [
        create_lq_df(pd.concat([x,creative_all_state['not_creative']],axis=1)).drop('not_creative',axis=1) for
        x in [subsector_df,creative_df]]

    #Now we combine the subsector and industry data for mapping

    creative_state_lq = pd.concat([creative_industries_state_lq,creative_subsector_state_lq],axis=1)

    creative_state_lq.rename(columns={0:'all_creative'},inplace=True)

    return(creative_state_lq)

In [ ]:
#We create a wide table with creative company counts by state and creative sector

#This simply pivots a grouped df where we counted the number of unique ids in every state-sector combination
creative_subsector_state = pd.pivot_table(
    cis.groupby(['state','creative_sector'])['id'].count().reset_index(drop=False),
    index='state',
    columns='creative_sector',values='id')

creative_subsector_state.head()

#Also calculate the total CIs per state, summing over the rows above
creative_industries_state =creative_subsector_state.sum(axis=1)

In [ ]:
#And now we combine this with the total counts by sector

#All state counts
#Concatenate and sum the company counts we extracted before
all_state = pd.concat([x[2][0] for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1)

#Make the index lowercase for merging
#We also need to remove trailing whitespaces 
all_state.index = [x.lower().strip() for x in all_state.index]

creative_state_lq = process_counts_to_get_lqs(creative_subsector_state,creative_industries_state,all_state)

In [ ]:
creative_state_lq.head()

In [ ]:
state_creative_lqs_polys = pd.merge(estados,creative_state_lq.reset_index(drop=False),left_on='ESTADO',
                               right_on='state')

In [ ]:
def plot_map(polygon_df,sector,discretised=False,ax=ax):
    '''
    This function takes a polygon df and a variable and returns a map. We can ask it to discretise the data
    into deciles too. 
    
    '''
    
    #If we want to discretise the data we transform the sector variable into deciles
    if discretised!=False:
        polygon_df[sector]= pd.qcut(polygon_df[sector],q=np.arange(0,1.1,0.1),
                                    labels=False,duplicates='drop')
        
    polygon_df.plot(sector,ax=ax,cmap='bwr',legend=False,edgecolor='black',
                                 linewidth=0.5)

    ax.set_title('{x}'.format(x=sector),size=16)
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
#And then we map
#NB We haven't added legends yet - 
#the graphs are not strictly comparable because the color-scale is set based on the distribution for each variable

fig,ax = plt.subplots(figsize=(8,12),nrows=5,ncols=2)

for num,sector in enumerate(creative_state_lq.columns):
    
    if num<5:
        col=0
        row=num
    else:
        col=1
        row=num-5
    
    plot_map(polygon_df=state_creative_lqs_polys,sector=sector,ax=ax[row][col])
    
fig.suptitle('Creative business specialisation by state',y=1.02,size=18)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_business_spec_maps.png'.format(date=today_str)),bbox_inches='tight')


In [ ]:
# Need to do something similar with employment.

#Create employment LQs
#Get employment estimates by subsector
cr_subsector_employment_state = pd.pivot_table(cis.groupby(
    ['state','creative_sector'])['employment_estimate'].sum().reset_index(drop=False),
                                               index='state',
                                               columns='creative_sector',
                                               values='employment_estimate')

#Get creative industries employment
all_creative_employment_state = cr_subsector_employment_state.sum(axis=1)

#We would also need to get employment estimates by location
all_sectors_employment_state = pd.concat(
    [get_state_employment_estimates(x[2][4]) for x in denue_eda_outputs],axis=1,sort=True).sum(axis=1)

all_sectors_employment_state.index = [x.lower().strip() for x in all_sectors_employment_state.index]

#Same as we did before, using the function to process creative dfs into lqs
creative_state_lq_emp =process_counts_to_get_lqs(cr_subsector_employment_state,all_creative_employment_state,
                                                all_sectors_employment_state)

#And merge with polygons for mapping
state_creative_emp_lqs_polys = pd.merge(estados,creative_state_lq_emp.reset_index(drop=False),left_on='ESTADO',
                               right_on='state')



In [ ]:
#And then we map.
#Still no legend

fig,ax = plt.subplots(figsize=(8,12),nrows=5,ncols=2)

for num,sector in enumerate(creative_state_lq_emp.columns):
    
    if num<5:
        col=0
        row=num
    else:
        col=1
        row=num-5
    
    plot_map(polygon_df=state_creative_emp_lqs_polys,sector=sector,ax=ax[row][col])
    
fig.suptitle('Creative employment specialisation by state',y=1.02,size=18)

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_empl_spec_maps.png'.format(date=today_str)),bbox_inches='tight')


A lot of concentration in Mexico DF and Nuevo León with some exceptions like Film Radio and TV in the North or Durango and Guerrero in Crafts

In [ ]:
#Here we are discretising each LQ distribution we plot (bin it in its decile)
#Colours for states are comparable (everywhere is binned into deciles)

fig,ax = plt.subplots(figsize=(8,12),nrows=5,ncols=2)

for num,sector in enumerate(creative_state_lq_emp.columns):
    
    if num<5:
        col=0
        row=num
    else:
        col=1
        row=num-5
    
    plot_map(polygon_df=state_creative_emp_lqs_polys,sector=sector,ax=ax[row][col],discretised=True)
    
fig.suptitle('Creative employment specialisation by state (discretised)',y=1.02,size=18)

plt.tight_layout()


Interesting - does this suggest some trans-state clusters?

### Co-location

In [ ]:
def plot_colocation(lq_df,name_for_title,ax=ax):
    '''
    Function that creates a co-location heatmap taking a df with LQs by location (state) as input
    
    '''
    #Create the correlation matrix with the input df
    correlation_matrix = lq_df.corr()
    
    #Plot
    im = ax.imshow(correlation_matrix,cmap='seismic',aspect='auto')
    
    #Colorbar
    fig.colorbar(im,ax=ax)
    
    #Axes
    ax.set_xticks(np.arange(len(correlation_matrix)))
    ax.set_yticks(np.arange(len(correlation_matrix)))
    
    ax.set_xticklabels(correlation_matrix.index,rotation=45,ha='right',size=14)
    ax.set_yticklabels(correlation_matrix.index,size=14)
    
    ax.set_title('{name} co-location'.format(name=name_for_title),size=14)
    return(ax)
    
    
    

In [ ]:
fig,ax = plt.subplots(figsize=(8,12),nrows=2,sharex=True)

sectors_coloc = ['advertising_marketing','design','software','architecture',
                 'music_performing_arts','publishing',
                 'film_radio_tv','libraries_museums','crafts']


plot_colocation(creative_state_lq[sectors_coloc],'Business',ax=ax[0])
plot_colocation(creative_state_lq_emp[sectors_coloc],'Employment',ax=ax[1])

plt.tight_layout()

plt.savefig(os.path.join(fig_dir,'{date}_co_location.pdf'.format(date=today_str)))

In [ ]:
def co_location_heatmap(lq_df,name_for_title,ax=ax):
    '''
    Function that creates a co-location heatmap taking a df with LQs by location (state) as input
    
    '''
    #Create the correlation matrix with the input df
    correlation_matrix = lq_df.corr()
    
    #Plot
    im = ax.imshow(correlation_matrix,cmap='seismic',aspect='auto')
    
    #Colorbar
    fig.colorbar(im,ax=ax)
    
    #Axes
    ax.set_xticks(np.arange(len(correlation_matrix)))
    ax.set_yticks(np.arange(len(correlation_matrix)))
    
    ax.set_ytickslabels(correlation_matrix.index)
    ax.set_xtickslabels(correlation_matrix.index)
    
    ax.set_title('{name} co-location'.format(name=name_for_title))
    return(ax)
    
    
    

## Contact detail availability

To conclude our initial exploration of DENUE data, we have a look at the availability of contact details for companies.

* How many have addresses (we will simply count postcodes)
* How many have telephones?
* How many have email addresses?


In [ ]:
#We import product to calculate the cartesian product of variables

from itertools import product

In [ ]:
#We will create a reduced dataframe with the variables we are interested in: 
#(contact and company characteristics relevant for the survwt)

#Select contact variables

#Contact variables
contact_vars = ['postcode','telephone','email','website']

#Metadata for analysis
metadata_vars = ['employee_sizeband','creative_sector','state']

#Create reduced df
contact_cis = cis[contact_vars+metadata_vars]

In [ ]:
#How many have postcode, telephone and email?

contact_cis[contact_vars].apply(lambda x: x.isna()==False).mean()

#We have postal addresses for everyone, telephone for 55% and email for 22%

In [ ]:
#This list comprehension creates a dict where the keys are the variables we are considering and the 
#values are the cross-tabs between each category and availability of contact details.
# We don't consider postal addresses because those are available for almost everyone.

cross_tabs = {x+'_'+y:pd.crosstab(contact_cis[x],contact_cis[y].isna()==False,normalize=0) for x,y in 
              product(metadata_vars,['telephone','email'])}



In [ ]:
#Plot contact details

fig,ax = plt.subplots(nrows=3,ncols=2,figsize=(11,12),gridspec_kw={'height_ratios':[2,2,5]},sharex='col')

#Go through the metadata variables we are interested in (eg sizeband, sector, state)    
for num,x in enumerate(metadata_vars):
    
    #Select the right cross tabs
    x = [v for k,v in cross_tabs.items() if x in k]
    
    #Loop over them
    for ct in x:
        #We make sure everything comes out in the right order by checking the name of the contact variable
        
        
        #The telephone crosstabs will always be in the left column
        if ct.columns.name=='telephone':
            
            #NB we are sorting the rows so that the places with most availability are at the top
            ct.sort_values(True,ascending=True).iloc[:,[1,0]].plot.barh(ax=ax[num][0],stacked=True)
                
            #We add a vertical line in the 50%
            ax[num][0].vlines(x=0.5,ymin=0,ymax=len(ct),linestyle=':')
    
        #The email crosstabs will always be in the right column
        else:
            ct.sort_values(True,ascending=True).iloc[:,[1,0]].plot.barh(ax=ax[num][1],stacked=True)
            
            ax[num][1].vlines(x=0.5,ymin=0,ymax=len(ct),linestyle=':')
    
#Thig layout    
plt.tight_layout()

#name
fig.suptitle('Availability contact details by Company size / sector / State',y=1.02,size=18)

plt.savefig(os.path.join(fig_dir,'{date}_contact_details_summary'.format(date=today_str)),bbox_inches='tight')

In [ ]:
#We conclude with a couple of heatmaps
contact_cis['has_telephone'] = contact_cis.telephone.isna()==False
contact_cis['has_email'] = contact_cis.email.isna()==False


contact_cross = [pd.pivot_table(
    contact_cis.groupby(['creative_sector','state'])[x].mean().reset_index(drop=False),
    index='creative_sector',columns='state',values=x) for x in ['has_telephone','has_email']]


In [ ]:
def make_hm(ct,ax=ax):
    '''
    Takes a crosstab and returns a heatmap
    
    '''
    #Extracts the sectors and states sorted
    
    sectors_contact_sorted = ct.mean(axis=1).sort_values(ascending=False).index
    states_contact_sorted = ct.mean(axis=0).sort_values(ascending=False).index

    #Plots
    ims = ax.imshow(ct.loc[sectors_contact_sorted,states_contact_sorted],cmap='seismic',aspect='auto',
                   vmax=1,vmin=0)

    #Creates the axis labels 
    ax.set_yticks(np.arange(len(ct.index)))
    ax.set_yticklabels(sectors_contact_sorted,size=12)
    ax.set_xticks(np.arange(len(ct.columns)))
    ax.set_xticklabels(states_contact_sorted,rotation=45,ha='right',size=12)
    
    #Draw the color bar
    fig.colorbar(ims,ax=ax)

    return(ims)
    

In [ ]:
fig,ax = plt.subplots(figsize=(16,9),nrows=2)

make_hm(contact_cross[0],ax=ax[0])
make_hm(contact_cross[1],ax=ax[1])

plt.tight_layout()

ax[0].set_title('Contact availability: Telephone by state and sector',size=14)
ax[1].set_title('Contact availability: Email by state and sector',size=14)


plt.savefig(os.path.join(fig_dir,'{date}_state_sector_availability.pdf'.format(date=today_str)),bbox_inches='tight')

### And what about the websites?

In [ ]:
contact_cis['has_website'] =contact_cis['website'].isna()==False

In [ ]:
fig,ax = plt.subplots(figsize=(10,6))


pd.pivot_table(
    contact_cis.groupby(
        ['creative_sector','employee_sizeband'])['has_website'].mean().reset_index(drop=False),
    index='creative_sector',columns='employee_sizeband',
    values='has_website').T.loc[sizeb_sorted,:].plot.bar(ax=ax,width=0.8,title='Website availability by sector & size')

ax.legend(ncol=2)
